In [10]:
from huggingface_hub import notebook_login
notebook_login()


In [ ]:
# 1. Install required libraries
!pip install -q transformers datasets peft accelerate bitsandbytes trl

# 2. Import libraries
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer

# 3. Load CodeAlpaca dataset
dataset = load_dataset("yahma/alpaca-cleaned")

# 4. Format dataset for SFT
def format_codealpaca(example):
    prompt = f"### Instruction:\n{example['instruction']}\n\n### Input:\n{example['input']}\n\n### Response:\n{example['output']}"
    return {"text": prompt}

dataset = dataset.map(format_codealpaca)
dataset = dataset.remove_columns(['instruction', 'input', 'output'])

# 5. Load tokenizer and model in 4-bit
model_name = "codellama/CodeLlama-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # prevent padding issues

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto"
)

# 6. Prepare model for LoRA
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

# 7. Define training arguments
training_args = TrainingArguments(
    output_dir="./codealpaca-codellama-7b",
    num_train_epochs=2,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    logging_steps=10,
    save_strategy="epoch",
    fp16=True,
    optim="paged_adamw_8bit",
    report_to="none",
    push_to_hub=True,  # Will push after training
)

# 8. SFT Trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    processing_class=tokenizer,  # use `tokenizer`, not `processing_class`
    args=training_args
)
# 9. Train the model
trainer.train()

# 10. Push to Hugging Face
trainer.push_to_hub("code-assistant-llama7b")


Map:   0%|          | 0/51760 [00:00<?, ? examples/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Adding EOS to train dataset:   0%|          | 0/51760 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/51760 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/51760 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:457: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default)

Step,Training Loss
10,1.662200
20,1.299700
30,1.240500
40,1.134100
50,1.269100
60,1.164800
70,1.223900
80,1.165300
90,1.240800
100,1.229100


Step,Training Loss
10,1.662200
20,1.299700
30,1.240500
40,1.134100
50,1.269100
60,1.164800
70,1.223900
80,1.165300
90,1.240800
100,1.229100
